In [95]:
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import RMSprop

In [96]:
class CNN:
    def __init__(self):
        pass
    
    def setup(self, input_shape=(54, 43, 2), n_filters=[10], kernel_size=[[5, 5]], padding='valid', activation='relu', n_neurons=10, activation_dense='relu'):
        
        Input = K.Input(shape=input_shape)
        
        tmp = []
        tmp.append(Input)
        for f in range(len(n_filters)):
            conv = Conv2D(filters=n_filters[f], kernel_size=kernel_size[f], padding=padding, activation=activation)(tmp[-1])
            tmp.append(conv)
        
        flat = Flatten()(tmp[-1])
        
        dense = Dense(n_neurons, activation_dense)(flat)
        out = Dense(units=3, activation='softmax')(dense)
        
        model = K.Model(inputs=Input, outputs=out)
        
        return model
        

In [97]:
path = '/p/project/training2005/HZG_Challenge/'

temperature_data = np.load(path+'tas_train.npy')
pressure_data = np.load(path+'psl_train.npy')


x_train_full = np.concatenate([temperature_data, pressure_data], axis=1)


nao_data = np.load(path+'nao_index_train.npy')

### Categorize NAO into 3 categories #####
med_ids = np.where(np.logical_and(-1.5<nao_data, nao_data<1.5))[0]
nao_data[nao_data>=1.5] = 1. 
nao_data[nao_data<=-1.5] = -1.
nao_data[med_ids] = 0.

### One hot encoding of targets ###
one_hot = np.zeros((len(nao_data), 3))
one_hot[nao_data==1.] = [1, 0, 0]
one_hot[nao_data==-1.] = [0, 0, 1]
one_hot[nao_data==0.] = [0, 1, 0]
print(nao_data[:5])
print(one_hot[:5])


[1. 0. 1. 0. 0.]
[[1. 0. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [98]:
NN = CNN()

In [99]:
temp_data = temperature_data.reshape((temperature_data.shape[0], 54, 43))
press_data = pressure_data.reshape((pressure_data.shape[0], 54, 43))

In [100]:
stacked_data = np.array([temp_data, press_data])
stacked_data = stacked_data.reshape((stacked_data.shape[1], stacked_data.shape[2], stacked_data.shape[3], stacked_data.shape[0]))
print(stacked_data.shape)

(900, 54, 43, 2)


In [101]:
mod = NN.setup(input_shape=stacked_data.shape[1:])


In [103]:
mod.compile(loss=categorical_crossentropy, optimizer=RMSprop())

In [ ]:
## train model ####

history = mod.fit(stacked_data, one_hot, batch_size=30, epochs=5, validation_split=0.05)

Train on 855 samples, validate on 45 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
855/855 [==============================] - 5s 6ms/sample - loss: 1.0003 - val_loss: 0.7618
Epoch 2/5
855/855 [==============================] - 3s 3ms/sample - loss: 0.5019 - val_loss: 0.7042
Epoch 3/5
390/855 [============>.................] - ETA: 1s - loss: 0.4066